In [69]:
import numpy as np

# Linear Decision boundaries
- Logistic Regression

In [70]:
def sigmoid(z):
    #return 1 / (1- np.exp(z))
    return 1/(1+np.exp(-z))

In [71]:
x_train = np.array([0., 1, 2, 3, 4, 5])
y_train = np.array([0,  0, 0, 1, 1, 1])

w_in = np.zeros((1))
b_in = 0

In [87]:
threshold=0.5
def predict(x, w, b):
    pred_y=np.zeros(len(x))
    for i, _x in enumerate(x):
        z= np.dot(w, _x)+b
        pred_y[i]= sigmoid(z)
    return np.round(pred_y)

print(f'y:{y_train}\n _y:{predict(x_train, w_in, b_in)}')

y:[0 0 0 1 1 1]
 _y:[0. 0. 0. 0. 0. 0.]


/var/folders/41/615730cj21lfyfd4hqhcdcbr0000gn/T/ipykernel_3413/3537664300.py:6: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pred_y[i]= sigmoid(z)


# Non-linear Decision boundaries
- Polinomial Logistic Regression

In [73]:
x_train2 = np.c_[x_train, x_train**2]
x_train2.shape

(6, 2)

In [74]:
w_in2 = np.zeros((2))
b_in2 = 0

In [75]:
threshold=0.5
def predict(x, w, b):
    z= np.dot(x, w) + b
    pred_y= sigmoid(z) 
#     pred_y=np.zeros(len(x))
#     for i, _x in enumerate(x):
#         z= np.dot(w, _x)+b
#         pred_y[i]= sigmoid(z)
    return np.array([0 if threshold <= _y else 1 for _y in pred_y])

print(f'y:{y_train}\n _y:{predict(x_train2, w_in2, b_in2)}')

y:[0 0 0 1 1 1]
 _y:[0 0 0 0 0 0]


# Cost Function
- 선형회귀에서 사용한 MSE는 로지스틱 회귀에서는 non-convex(local minima 짱 많음)
- BCE
    - 실제 레이블이 1이면 -log(pred_y)
    - 실제 레이블이 0이멸 -log(1-pred_y)
    - 조건으로 안 나누고 '-y_true*np.log(y_hat)-(1-y_true)*np.log(1-y_hat)'하면 동일함

In [79]:
def cost(y, pred_y):
    m = len(y)
    epsilon = 1e-15  # Small value to avoid log(0)
    pred_y = np.clip(pred_y, epsilon, 1 - epsilon)  # Clip predictions
    loss= np.sum([-y_true*np.log(y_hat)-(1.-y_true)*np.log(1.-y_hat) for y_hat, y_true in zip(pred_y, y)])
    #loss =np.sum([-np.log(y_hat) if y_true else -np.log(1-y_hat) for y_hat, y_true in zip(pred_y, y)])
#     loss=0
#     for y_hat, y_true in zip(pred_y, y):
#         error = -y_true*np.log(y_hat)-(1-y_true)*np.log(1-y_hat)
#         loss = loss + error
#         print(f'y_hat:{y_hat}, y_true:{y_true}, error:{error}, loss:{loss}')
    loss=  loss / m
    return loss

pred_y=predict(x_train2, w_in2, b_in2)
print(f'cost:{cost(y_train, pred_y)}')

cost:17.269388197455342


# Gradient Descent w Regularization
- dw = 1/m*sum(y_hat-y_true)*x
- db = 1/m*sum(y_hat-y_true)
- https://www.coursera.org/learn/machine-learning/ungradedLab/KtxCL/optional-lab-gradient-descent-for-logistic-regression/lab?path=%2Fnotebooks%2FC1_W3_Lab06_Gradient_Descent_Soln.ipynb
- regul/2m * sum(w)
- https://www.coursera.org/learn/machine-learning/ungradedLab/36A9A/optional-lab-regularization/lab?path=%2Fnotebooks%2FC1_W3_Lab09_Regularization_Soln.ipynb

In [89]:
def train(x, y, epoch=10, lr= 1e-3):
    n_samples, n_features= x.shape
    w = np.zeros((n_features))
    b = 0
    epsilon = 1e-15  
    for _ in range(epoch):
        z= np.dot(x, w) + b
        #pred_y= np.round(sigmoid(z)) 
        pred_y = np.clip(sigmoid(z), epsilon, 1 - epsilon)
        #dw = np.sum(np.dot((pred_y-y).T,x))
        dw = np.dot((pred_y-y).T,x)/ n_samples
        db = np.sum(pred_y-y) / n_samples
        w = w - lr*dw
        b = b - lr*db
#         print(f'w:{w}, b:{b}')
#     print(f'w:{w}, b:{b}')
    return w, b

w, b =train(x_train2, y_train)
pred_y=predict(x_train2,w, b)
print(f'cost:{cost(y_train, pred_y)}')

cost:11.51319199744696


In [90]:
def train(x, y, epoch=10, lr= 1e-3, regl=1e-5):
    n_samples, n_features= x.shape
    w = np.zeros((n_features))
    b = 0
    epsilon = 1e-15  
    for _ in range(epoch):
        z= np.dot(x, w) + b
        pred_y= sigmoid(z)
        pred_y = np.clip(sigmoid(z), epsilon, 1 - epsilon)
        dw =(np.dot((pred_y-y).T,x)+ regl*w)/n_samples
        db = np.sum(pred_y-y)/n_samples
        w = w - lr*dw
        b = b - lr*db
#         print(f'w:{w}, b:{b}')
#     print(f'w:{w}, b:{b}')
    return w, b

w, b =train(x_train2, y_train)
pred_y=predict(x_train2,w, b)
print(f'cost:{cost(y_train, pred_y)}')

cost:11.51319199744696


# Sklearn
- https://www.coursera.org/learn/machine-learning/ungradedLab/F3ZpI/optional-lab-logistic-regression-with-scikit-learn/lab?path=%2Fnotebooks%2FC1_W3_Lab07_Scikit_Learn_Soln.ipynb
- https://velog.io/@gayeon/%EB%8D%B0%EC%9D%B4%ED%84%B0-%EB%B6%84%EC%84%9D-%EC%B4%88%EB%B3%B4%EC%9E%90%EB%A5%BC-%EC%9C%84%ED%95%9C-Logistic-Regression-with-Scikit-Learn

In [62]:
from sklearn.linear_model import LogisticRegression

# 모델 생성 및 학습 시키기
logistic = LogisticRegression()
logistic.fit(x_train2, y_train)

# 결과 확인
logistic.score(x_train2, y_train)

validation 데이터셋 정확도


1.0